# Training

In [3]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.chdir('/content/drive/MyDrive/LearningToCountEverything_new')

In [6]:
# Import Everything
import torch.nn as nn
from model import *
from utils_ltce import MAPS, Scales, Transform,TransformTrain,extract_features, visualize_output_and_save
from PIL import Image
import os
import torch
import argparse
import json
import numpy as np
from tqdm import tqdm
from os.path import exists,join
import random
import torch.optim as optim
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
import torchvision.ops.boxes as bops
from funcs import *

In [7]:
# Flag to enable GPU or not
enable_gpu = True
if enable_gpu:
  print("=====> GPU")
else:
  print("=====> CPU")

=====> GPU


In [8]:
# Setting relevant paths
data_path = '/content/drive/MyDrive/LearningToCountEverything_new/data/'
image_path = '/content/drive/MyDrive/Colab Notebooks/LearningToCountEverything/data/'
output_dir = "./logsSave"
check_dir = "./checkpoints"
test_split = "train" #choices=["train", "test", "val"]
gpu_id = 0 
learning_rate = 1e-5
anno_file = image_path + 'annotation_FSC147_384.json'
data_split_file = image_path + 'Train_Test_Val_FSC_147.json'
im_dir = image_path + 'images_384_VarV2'
gt_dir = image_path + 'gt_density_map_adaptive_384_VarV2'

# Choosing the specific Regressor Class 
# (our --> CountRegressorAML)
# (baseline --> CountRegressor)
regressor = CountRegressorAML(6, pool='mean')

# Setting initial weights
weights_normal_init(regressor, dev=0.001)

# Backbone for feature extraction
# Resnet50FPN
# VGG16
backbone_model = Resnet50FPN()

# Setting parameters and variables accordingly based on use of GPU or CPU
if enable_gpu: 
    regressor.cuda()
    regressor.train()
    criterion = nn.MSELoss().cuda()
    backbone_model.cuda()
    backbone_model.eval()
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
else: 
    regressor.train()
    criterion = nn.MSELoss()
    backbone_model.eval()

# Trainig Optimizer
optimizer = optim.Adam(regressor.parameters(), lr = learning_rate)

if not exists(output_dir):
    os.mkdir(output_dir)

if not exists(check_dir):
    os.mkdir(check_dir)  

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [9]:
with open(anno_file) as f:
    annotations = json.load(f)

with open(data_split_file) as f:
    data_split = json.load(f)

In [10]:
# Load pretrained YOLO Model
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-12-13 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)



  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


In [11]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
# Most relevant parameters to be considered are:
# - epochs: number of epochs for training phase
# - yolo_flag: Whether we want to enhance additional boxes through the pre-trained yolo model
# - yolo_threshold: If it is an integer take max that value of boxes, if it is a probaility, take all boxes with a probabilty higher than that
# - load_checkpoint: Used to load a specific checkpoint
run_train_phase(epochs=40, backbone_model=backbone_model, regressor=regressor, yolo_model=model_yolo,
                optimizer=optimizer, criterion=criterion, annotations=annotations, 
                data_train=data_split['train'], shuffle=True, data_val=data_split['val'], 
                num_img_train=len(data_split['train']), num_img_val=len(data_split['val']), 
                yolo_flag=True, yolo_threshold=3, 
                plot_flag=False, 
                save='/content/drive/MyDrive/LearningToCountEverything_new/data/pretainedModelsOurs/model_resnet_yolo.pth', 
                im_dir=im_dir, gt_dir=gt_dir,
                enable_gpu=enable_gpu, augment=True, 
                checkpoint_dir='checkpoints/', 
                #load_checkpoint=False)
                load_checkpoint='/content/drive/MyDrive/LearningToCountEverything_new/checkpoints/checkpoint14.pt')

# Testing

In [ ]:
import torch.nn as nn
from model import Resnet50FPN, CountRegressor, weights_normal_init, weights_xavier_init
from utils_ltce import MAPS, Scales, Transform,TransformTrain,extract_features, visualize_output_and_save
from PIL import Image
import os
import torch
import argparse
import json
import numpy as np
from tqdm import tqdm
from os.path import exists,join
import random
import torch.optim as optim
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
import torchvision.ops.boxes as bops
from funcs import *

In [ ]:
data_path = '/content/drive/MyDrive/LearningToCountEverything_new/data/'
output_dir = "./logsSave"
test_split = "test" #choices=["train", "test", "val"]
gpu = 0 
learning_rate = 1e-5
anno_file = data_path + 'annotation_FSC147_384.json'
data_split_file = data_path + 'Train_Test_Val_FSC_147.json'
im_dir = data_path + 'images_384_VarV2'
gt_dir = data_path + 'gt_density_map_adaptive_384_VarV2'


regressor = CountRegressorAML(6, pool='mean')

# Optimizer will not be used, but needed to load a checkpoint...
optimizer = optim.Adam(regressor.parameters(), lr = learning_rate)

backbone_model = Resnet50FPN()

if enable_gpu: 

  regressor.cuda()
  regressor.eval()
  criterion = nn.MSELoss().cuda()
  backbone_model.cuda()
  backbone_model.eval()

else: 
  regressor.train()
  criterion = nn.MSELoss()
  backbone_model.eval()

optimizer = optim.Adam(regressor.parameters(), lr = learning_rate)

if not exists(output_dir):
    os.mkdir(output_dir)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)

In [ ]:
with open(anno_file) as f:
    annotations = json.load(f)

with open(data_split_file) as f:
    data_split = json.load(f)

In [ ]:
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-12-12 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
res = test(data=data_split['test'], 
          #num_img=2,
          num_img=len(data_split['test']), 
          backbone_model=backbone_model, regressor=regressor, optimizer=optimizer,
          yolo_model=model_yolo, yolo_flag=True, yolo_threshold=3, annotations=annotations, 
          plot_flag=False, im_dir=im_dir, enable_gpu=enable_gpu, 
          model_path=best_model_dir,
          adapt=False, gradient_steps=100, learning_rate=1e-7)